# Train and evaluate models

This notebook is meant to standardize model training/evaluation for the base models of the experiments.

It goes through
1. Evaluating saved models on saved dataset
2. Training new models on new dataset

## TODO

Per model, per layer:
* for each variable, can we find it?
* induce combination of variables that CAN'T be separated

In [ ]:
!pip install -r requirements.txt

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import numpy as np
import torch
import sys
sys.path.append('../utils')

random.seed(0)
np.random.seed(0)
_ = torch.manual_seed(0)
# _ = torch.cuda.manual_seed(42) # only if using GPU

## 1. Evaluating saved models

For now, we create a new test dataset, because the `images.npy` file doesn't contain the labels (circularity, color, area) for the images.

In [5]:
# toggle i to select model to evaluate, toggle n_test to select number of test samples
i = 1
n_test = 1000

Not very high accuracy... Maybe we need to check the tensorflow -> pytorch conversion?

In [7]:
from model_utils import PyTorchCNN, evaluate
from data_utils import create_dataset

# load model
model = PyTorchCNN()
model.load_state_dict(torch.load(f'../pytorch_models/cnn_model_{i}.pth'))

# create our own evaluation dataset
images, labels = create_dataset(n_test)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X = torch.tensor(images.reshape((-1, 1, 28, 28))).float()
y = torch.tensor(np.matmul(labels, coefficients) > 0.6).float()

# evaluate model
accuracy = evaluate(model, X, y)
print(f'Accuracy: {accuracy:.4f}')

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 40.13it/s]

Accuracy: 0.9660


In [28]:
trained_models[0][0].summary()

Model: "functional_200"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ keras_tensor_1800 (InputLayer)  │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer1 (Conv2D)                 │ (None, 28, 28, 16)     │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_400               │ (None, 14, 14, 16)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer2 (Conv2D)                 │ (None, 14, 14, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_401               │ (None, 7, 7, 32)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer3 (Conv2D)                 │ (None, 7, 7, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_200 (Flatten)           │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 3)              │         9,411 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_200 (Dense)               │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 98,135 (383.34 KB)

 Trainable params: 32,711 (127.78 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 65,424 (255.57 KB)

In [16]:
X_tf = images.reshape((-1, 28, 28, 1))

preds = trained_models[0][0].predict(X_tf)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [26]:
(preds.squeeze().round() == y.numpy()).mean()

0.966

## 2. Training new models

I'm less familiar with tensorflow, so I created a simple function for training a pytorch model on the ellipse data.

In [5]:
# toggle n_train and n_test to select number of training/evaluation samples
n_train = 50000
n_test = 1000

In [6]:
from model_utils import PyTorchCNN, train

# create our own training dataset
images_train, labels_train = create_dataset(n_train)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X_train = torch.tensor(images_train.reshape((-1, 1, 28, 28))).float()
y_train = torch.tensor(np.matmul(labels_train, coefficients) > 0.6).float()

model = PyTorchCNN()
train(model, X_train, y_train, lr=0.0001, num_epochs=5)

Training (Epoch 5): 100%|██████████| 196/196 [00:09<00:00, 20.51it/s, loss=0.193]


The evaluation accuracy looks good! This suggests to me that maybe we're not loading the tensorflow weights correctly? Not sure...

In [7]:
from model_utils import evaluate

# create our own evaluation dataset
images_test, labels_test = create_dataset(n_test)
coefficients = np.array([0.4, 0.4, 0.4])

# batch, channel, height, width
X_test = torch.tensor(images_test.reshape((-1, 1, 28, 28))).float()
y_test = torch.tensor(np.matmul(labels_test, coefficients) > 0.6).float()

accuracy = evaluate(model, X_test, y_test)
print(f'Accuracy: {accuracy:.4f}')

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 45.20it/s]

Accuracy: 0.9100


We'll use this model for the rest of our analysis

In [8]:
torch.save(model.state_dict(), f'pytorch_models/amir_cnn_model.pth')

In [9]:
model = PyTorchCNN()
model.load_state_dict(torch.load(f'pytorch_models/amir_cnn_model.pth'))

evaluate(model, X_test, y_test)

Evaluating: 100%|██████████| 4/4 [00:00<00:00, 48.11it/s]


0.9100000262260437

## Convert Tensorflow to Pytorch

In [3]:
from model_utils import PyTorchCNN

def keras_to_pytorch_cnn(keras_model):
    pytorch_model = PyTorchCNN()

    # Copy weights
    pytorch_model.conv1.weight.data = torch.FloatTensor(keras_model.get_layer('layer1').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv1.bias.data = torch.FloatTensor(keras_model.get_layer('layer1').get_weights()[1])

    pytorch_model.conv2.weight.data = torch.FloatTensor(keras_model.get_layer('layer2').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv2.bias.data = torch.FloatTensor(keras_model.get_layer('layer2').get_weights()[1])

    pytorch_model.conv3.weight.data = torch.FloatTensor(keras_model.get_layer('layer3').get_weights()[0].transpose(3, 2, 0, 1))
    pytorch_model.conv3.bias.data = torch.FloatTensor(keras_model.get_layer('layer3').get_weights()[1])

    pytorch_model.fc1.weight.data = torch.FloatTensor(keras_model.get_layer('dense1').get_weights()[0].T)
    pytorch_model.fc1.bias.data = torch.FloatTensor(keras_model.get_layer('dense1').get_weights()[1])

    pytorch_model.fc2.weight.data = torch.FloatTensor(keras_model.get_layer('dense_200').get_weights()[0].T)
    pytorch_model.fc2.bias.data = torch.FloatTensor(keras_model.get_layer('dense_200').get_weights()[1])

    return pytorch_model

In [5]:
import numpy as np
from tensorflow import keras

trained_models = np.load('../trained_models.npy', allow_pickle=True)
trained_models.shape

(100, 3)

In [ ]:
# for run in range(trained_models.shape[0]):
#     for coef in range(trained_models.shape[1]):
#         keras_model = trained_models[run][coef]
#         pytorch_model = keras_to_pytorch_cnn(keras_model)

#         model_index = run * trained_models.shape[1] + coef + 1
#         torch.save(pytorch_model.state_dict(), f'pytorch_models/cnn_model_{model_index}.pth')